**Educational Attainment and Success**


---


Chieh-Huang Chen \
Melvin Cherian \
Michael LI \


---

Course: CSC 59970 - Introduction to Data Science \
Instructor: Grant M. Long
Project



**Goals for this Project**

1.  Apply Python to load, clean, and process data sets.
2.  Identify key patterns in a data set using computational analysis and statistical methods.
3.  Apply principles of statistical modeling and machine learning to data.
4.  Effectively explain, visualize, and communicate empirical findings.


---

**Our Questions**


*   What human development indicators strongly correlate with adult education level in countries?
*   What are significant factors from each major category from the human development dataset that determines adult education level in countries?
*   Which countries have a strong potential in increasing their education level within the next five years?

---

*Thoughts from Chieh (Remove this before submitting the final project)*

1. Use the dataset and find potential indicators that have high coorelations with Education Index (https://en.wikipedia.org/wiki/Education_Index), which is used to judge each countries' education level based on the number of years a child is expected to attend school or university and the average number of years of education received by people ages 25 and older.

2.  Find the potential indicators for each country and use those potential indicators to predict education index through linear regression.

3. Assumptions of Linear Regression \
a.  Data is linear in form. \
b.  Sample is random. \
c.  Error terms have constant variance (homoscedasticity). \
d.  Error terms have a mean of zero based on the observed data. \
e.  Predictors are independent (no multicollinearity). \
f.   Errors are normally distributed. \

4. How to satisfy these assumptions from #3? \
a. Check if the correlation of determination is greater than 0.7  \
b. The data that we collected represents the consenus of the entire country. \
c. There's a constant variance because it's yearly. \
d. Get the standard error of the array and see if it is similar. \
e. Use an algorithm to remove coorelations between predictors. \
f. Yes it is. \

---

*Thoughts from Melvin (Remove this before submitting the final project)*


---

*Thoughts from Michael (Remove this before submitting the final project)*

---





*Libraries*


*   Numpy
*   Pandas
*   Matplotlib



In [0]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

*Loading Data into Memory*

---

We loaded two datasets into memory: Adult Education Level (AEL) dataset and Human Development Indicators (HDI) dataset.

We specifically removed column 9999 from our dataset because it only shows the latest value from the data drawn within 1990 to 2017. In this analysis, we analyzed these indicators over time to predict the future of education level in each country.

In [0]:
# Loading Datasets on to DataFrames
indicators_url = 'https://github.com/chc5/education-indicator-analysis/raw/master/2018_all_indicators.csv'
education_url = 'https://github.com/chc5/education-indicator-analysis/raw/master/adult_education_level.csv'

inds_df = pd.read_csv(indicators_url)
ed_df = pd.read_csv(education_url)

inds_df = inds_df.drop(columns=['9999'])

Many of our HDIs only have zero or one recordings in the dataset in years between 1990 to 2017. Due to the lack of data over time, we decided to remove these indicators that has at most 2 data points in each row.

When we removed these indicators, it cleans up at least 28% of the HDI dataset.

In [32]:
# Many indicators from 1990 to 2017 that only have at most one or two recordings for a specific indicators.
# These recordings may or may not indicate growth over the years which will not contribute to our analysis.
# These indicators will be dropped.
# When threshold = 1, it removed 25636 - 20588 = 5048 / 25636 = 20% of the indicators
threshold = 2
drop_stack = set()
length_before = len(inds_df)
print("Number of rows in the indicators data frame before the cleanup:", length_before)
for index, row in inds_df.iterrows():
    if row['1990':].count() <= threshold:
        drop_stack.add(index)

inds_df = inds_df.drop(drop_stack)
length_after = len(inds_df)
print("Number of rows in the indicators data frame after the cleanup:", length_after)
print("% of rows deleted: {0:.2f}%".format((length_before - length_after) * 100 / length_after))


Number of rows in the indicators data frame before the cleanup: 25636
Number of rows in the indicators data frame after the cleanup: 19911
% of rows deleted: 28.75%


*Plotting Correlations for each country*

We have a method to retrieve each indicators for a country we specify for future uses.

In [0]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);
    
  
def plot_corr_country(df, country):
  # drop ids etc., get only rows with country, transpose
  df_corr = inds_df.drop(df.columns[[0, 1, 3]], axis=1)[df.country_name == country].T
  # fix columns
  df_corr.columns = df_corr.iloc[0]
  # drop first two rops
  df_corr = df_corr[2:]
  # df_corr = inds_df.drop(inds_df.columns[[0, 1, 3]], axis=1).drop(inds_df.columns[range(5,32)], axis=1)[inds_df.country_name == "Albania"]
  df_corr = df_corr.apply(pd.to_numeric)
#   plot_corr(df_corr, size=20)
  return df_corr

### How do we determine the strongest indicators to Education Index for each country?

**Education Index:** A standard created by United Nations that determines educational attainment for each country. \
**Formula:** MYSI + EYSI / 2 \
Mean Years of Schooling Index(MYSI): MYS / 15 \
Mean Years of Schooling(MYS): Average number of years of education received by people ages 25 and older. \
Expected Years of Schooling Index(EYSI): EYS / 18 \
Expected Years of Schooling(EYS): The number of years a child is expected to attend school, or university, including the years spent on repetition. \

*Source: https://en.wikipedia.org/wiki/Education_Index* 



---


We get the strongest correlations for each country by getting the correlation coefficient between the Education Index to every other indicator in the row for that country. By squaring the correlation coefficient, we get the correlation of determination (R^2) which tells us the percentage of the variance in our dependent variable that can be explained by our independent variable. In this case, we sorted the indicators based on the correlation of determination and remove all the indicators where its correlation of determination to the Education Index is less than 0.7.

We then dropped the indicators that are directly dependent to the Education Index based on the formula such as 'Education index', 'Expected years of schooling (years)' and 'Mean years of schooling (years)'.

In [0]:
# Get the strongest indicators towards the Education Index by getting the strongest correlation of determination (R^2)
# between every other indicators to the Education Index. 
def get_strongest_correlations(df_corr, indicator, num_indicators = 100, threshold=0.7):
    correlations = []
    for row in df_corr.corr():
        corr_determ = df_corr.corr()[row][indicator] ** 2
        if corr_determ >= threshold:
          correlations.append([row, corr_determ])
    
    correlations.sort(key = lambda corr: corr[1], reverse=True)
    return correlations[:num_indicators]

In [0]:
# Remove multicollinearity betweeen the potential indicators by checking the coorelations between the two indicators.
# If there is a coorelation between the potential indicators, then remove the indicator that is least correlated with
# the dependent variable out of the two.
def remove_multicollinearity(df, features, target, threshold=0.8):
  correlations = df.corr()
  # Remove the target in the features list if it is in there.
  temp = [f for f in features if f != target]
  i = 0
  while i < len(temp):
    redo = False
    for f in temp:
      if correlations[f][temp[i]] > threshold \
        and correlations[temp[i]][target] < correlations[f][target]:
        temp[i], temp[-1] = temp[-1], temp[i]
        temp.pop()
        redo = True
        break
    if not redo:
      i += 1
  return temp


In [0]:
# Map indicators to categories
def map_ind_to_categories(df):
  ind_map = {}
  mapped_inds = df.groupby(['dimension', 'indicator_name']).size().reset_index().rename(columns={0:'count'})
  for index, row in mapped_inds.iterrows():
    ind_map[row['indicator_name']] = row['dimension']
  return ind_map

def group_indicators(df, potential_indicators):
  ind_map = map_ind_to_categories(df)
  groups = {}
  features = []
  for p in potential_indicators:
    if ind_map[p[0]] not in groups:
      groups[ind_map[p[0]]] = [p]
    else:
      groups[ind_map[p[0]]].append(p)
  for g in groups:
    features.append(max(groups[g], key=lambda g: g[1])[0])
  return features

In [0]:
# Get potential indicators from countries
def get_potential_indicators(df, country, target, ignored_features=[], multicollinearity=False, grouped=False):
  country_df = plot_corr_country(df, country)
  if target not in country_df:
    print("{} does not have an {}".format(country, target))
    return []
  potential_indicators = get_strongest_correlations(country_df, target)
  potential_indicators = [p for p in potential_indicators if p[0] not in ignored_features]
  features = [p[0] for p in potential_indicators] 
  if not multicollinearity:
    features = remove_multicollinearity(country_df, features, target)
  if grouped:
    features = group_indicators(df, potential_indicators)
  return features


In [38]:
# Dropping 'Education index', 'Expected years of schooling (years)', 'Mean years of schooling (years)' and others because they are part of the formula
dep_ed_ind = ['Education index', 
                 'Expected years of schooling (years)', 
                 'Mean years of schooling (years)', 
                 # I don't know if the indicators below is dependent to the education index. They might tell an interesting story.
                 'Expected years of schooling, female (years)', 
                 'Mean years of schooling, female (years)',
                 'Expected years of schooling, male (years)',
                 'Expected years of schooling, male (years)',
                 'Mean years of schooling, male (years)',
                 'Inequality-adjusted education index',
                 'Human Development Index (HDI)'
                ]   

target = 'Education index'
top_indicators = {}
country_indicators = {}
country_names = inds_df.country_name.unique()
for c in country_names:
  features = get_potential_indicators(inds_df, c, target, dep_ed_ind, True, True)
  country_indicators[c] = features
  for f in features:
    if f in top_indicators:
      top_indicators[f] += 1
    else:
      top_indicators[f] = 1
features = get_potential_indicators(inds_df, 'United States', target, dep_ed_ind, True, True)
print(features)
sorted_top_indicators = sorted(top_indicators.items(), key=lambda kv: kv[1], reverse=True)
i = 1
print("\n Top Indicators Used for Each Country")
for ind in sorted_top_indicators:
  print("{}. {} \t Number of countries: {}".format(i, ind[0], ind[1]))
  i += 1


Korea (Democratic People's Rep. of) does not have an Education index
Somalia does not have an Education index
Marshall Islands does not have an Education index
Monaco does not have an Education index
Nauru does not have an Education index
San Marino does not have an Education index
Tuvalu does not have an Education index
['Population using improved drinking-water sources (%)', 'Current health expenditure (% of GDP)', 'Maternal mortality ratio (deaths per 100,000 live births)', 'Employment in services (% of total employment)', 'Forest area (% of total land area)', 'Population with at least some secondary education, female (% ages 25 and older)', 'Suicide rate, male (per 100,000 people)', 'Mobile phone subscriptions (per 100 people)', 'Total population (millions)', 'Exports and imports (% of GDP)', 'Gross domestic product (GDP), total (2011 PPP $ billions)']

 Top Indicators Used for Each Country
1. Working poor at PPP$3.10 a day (% of total employment) 	 Number of countries: 77
2. Fores

In [0]:
# print(inds_df.columns)
# for c in country_indicators:
#   country_df = inds_df.loc[inds_df['country_name'] == c]
#   target_df = country_df.loc[country_df['indicator_name'] == target]
#   target_df = target_df.drop(columns=['dimension', 'indicator_id', 'iso3', 'country_name'])
#   features_df = country_df.loc[country_df['indicator_name'].isin(country_indicators[c])]
#   features_df = features_df.drop(columns=['dimension', 'indicator_id', 'iso3', 'country_name']).T
#   print(features_df.columns)
#   X = features_df.values
#   Y = target_df.values
#   labelencoder = LabelEncoder()
#   X[:, 0] = labelencoder.fit_transform(X[:, 0])
#   onehotencoder = OneHotEncoder(categorical_features = [len(X)])
#   X = onehotencoder.fit_transform(X).toarray()
#   X = X[:, 1:]
#   print(features_df.dropna(inplace=True))


In [0]:
# Transpose the data frame to only have years and values as columns
def transpose_yearly_values(df):
  df = df.drop(columns=['dimension', 'indicator_id', 'iso3', 'country_name'])
  df = df.set_index('indicator_name').transpose()
  df['Year'] = pd.to_numeric(df.index)
  return df

# Predicting the missing values using Linear Regression Models
# and then filling out the missing values based on the model
# Preserve the Linear Regression for future uses
def get_filled_out_indicators(inds_df, country, indicators):
  country_df = inds_df.loc[inds_df['country_name'] == country]
  reg_inds = {}
  filled_indicators = {}
  for ind in indicators:
    ind_df = country_df.loc[country_df['indicator_name'] == ind]
    values_df = transpose_yearly_values(ind_df)
    # Get the missing values in each year and predict them later on.
    missing_years = values_df[values_df[ind].isnull()].Year.values
    values_df = values_df.dropna()
    # If there are nothing to fill out then we don't need to model it out.
    if missing_years.size <= 0:
        continue
    # Fit the data to the Linear Regression and predict the values
    X = values_df['Year'].values.reshape(-1, 1)
    Y = values_df[ind].values.reshape(-1, 1)
    reg = LinearRegression()
    reg.fit(X, Y)
    missing_years = missing_years.reshape(-1, 1)
    missing_values = reg.predict(missing_years)
    # Filling out the missing data back to values_df
    missing_years = missing_years.reshape(1, -1)[0]
    missing_values = missing_values.reshape(1, -1)[0]
    predicted_df = pd.DataFrame({'Year': missing_years, ind: missing_values})
    values_df = values_df.append(predicted_df, sort=False)
    # Preserve the filled out data and the regression for future uses
    reg_inds[ind] = reg
    filled_indicators[ind] = values_df
  return reg_inds, filled_indicators
# get_filled_out_indicators(inds_df, 'United States', ['Sex ratio at birth (male to female births)'])

In [82]:
regression_indicators, indicator_vals = get_filled_out_indicators(inds_df, 'United States', country_indicators['United States'])
country_df = inds_df.loc[inds_df['country_name'] == 'United States']
target_df = country_df.loc[country_df['indicator_name'] == target]
target_df = target_df.drop(columns=['dimension', 'indicator_id', 'iso3', 'country_name'])
target_df = target_df.set_index('indicator_name').transpose()
features_df = country_df.loc[country_df['indicator_name'].isin(country_indicators['United States'])]
features_df = features_df.drop(columns=['dimension', 'indicator_id', 'iso3', 'country_name'])
features_df = features_df.set_index('indicator_name').transpose()
features_df['Year'] = pd.to_numeric(features_df.index)
X = features_df.values
Y = target_df.values
reg = LinearRegression()
reg.fit(X, Y)
print(reg.score(X, Y))


{'Population using improved drinking-water sources (%)':       Population using improved drinking-water sources (%)  Year
2005                                          99.100000     2005
2010                                          99.200000     2010
2015                                          99.200000     2015
0                                             98.966667     1990
1                                             98.976667     1991
2                                             98.986667     1992
3                                             98.996667     1993
4                                             99.006667     1994
5                                             99.016667     1995
6                                             99.026667     1996
7                                             99.036667     1997
8                                             99.046667     1998
9                                             99.056667     1999
10                               

ValueError: ignored

In [0]:
def backtest(n_days):
    """
    n_days - amount of the last n_days that we whant to get prediction and calculate metrics
    """
    predictions = []
    true_values = []
    for i in reversed(range(1, n_days)):
        x_train = x[:len(x)-i]
        y_train = y[:len(y)-i]
        x_test = x[len(x)-i]
        y_test = y[len(y)-i]
        model = BayesianRidge()
        model.fit(x_train, y_train)
        predictions.append(model.predict([x_test])[0])
        true_values.append(y_test[0])
    return true_values, predictions